In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
home_dir = "../"

import pandas as pd
from utils.column_names_reader import get_col_names

In [2]:
force=False
if os.path.exists(home_dir+"data/gene/np_to_uniprot_mapping.csv") or force:
    np_to_uniprot_mapping_df = pd.read_csv(home_dir+"data/gene/np_to_uniprot_mapping.csv", sep="\t")
    print(np_to_uniprot_mapping_df.shape)
    print(np_to_uniprot_mapping_df.columns)
else:
    inp_filepath = home_dir+"data/gene/gene_refseq_uniprotkb_collab.gz"
    col_names = get_col_names(inp_filepath, "#NCBI_protein_accession")
    df_iterator = pd.read_csv(inp_filepath, compression='gzip', comment='#', chunksize=1000000, sep="\t", header=None, names=col_names) # cols: NCBI_protein_accession	UniProtKB_protein_accession
    # df = df_iterator.__next__()
    # print(df.shape)
    # df.head()

    df_list = []
    for i, df in enumerate(df_iterator):
        new_df = df[df["NCBI_protein_accession"].apply(lambda x: x.startswith("NP_"))]
        print(i, new_df.shape)
        df_list.append(new_df)
        # break

    np_to_uniprot_mapping_df = pd.concat(df_list)
    np_to_uniprot_mapping_df.shape

(1282967, 2)
Index(['NCBI_protein_accession', 'UniProtKB_protein_accession'], dtype='object')


In [3]:
np_to_uniprot_mapping_df["NCBI_protein_accession"].value_counts() # single NP-id is mapped to multiple uniprot-ids
np_to_uniprot_mapping_df[np_to_uniprot_mapping_df["NCBI_protein_accession"]=="NP_740526.1"] # same NP-id is mapped to multiple uniprot ids
# np_to_uniprot_mapping_df[np_to_uniprot_mapping_df["UniProtKB_protein_accession"]=="Q5SV97"] # S5G239 or Q5SV97, same uniprot id is mapped to multiple NP-ids
# np_to_uniprot_mapping_df[np_to_uniprot_mapping_df["NCBI_protein_accession"]=="NP_001164006.1"] # some NP-ids are not mapped
# np_to_uniprot_mapping_df.value_counts() # no duplications on pairs

NP_740526.1       368
NP_943823.1       212
NP_943793.1       209
NP_943796.1       199
NP_040978.1       195
                 ... 
NP_001294254.1      1
NP_001294254        1
NP_001294255        1
NP_001294256.1      1
NP_999999           1
Name: NCBI_protein_accession, Length: 963618, dtype: int64

In [26]:
updated_uniprot_ids_dict = {"S5G239": "Q5SV97",
                            "Q5T7M5": "Q96L58",
                            "Q1RMY8": "P62873",
                            "A0A024R056": "A0A140VJJ8",
                            "Q8N4N9": "O14764",
                            "Q5SYT7": "P12755"} # previous, new
for key, value in updated_uniprot_ids_dict.items():
    print(key, value)
    np_to_uniprot_mapping_df.loc[np_to_uniprot_mapping_df["UniProtKB_protein_accession"]==key, "UniProtKB_protein_accession"] = value

S5G239 Q5SV97
Q5T7M5 Q96L58
Q1RMY8 P62873
A0A024R056 A0A140VJJ8
Q8N4N9 O14764
Q5SYT7 P12755


In [27]:
np_to_uniprot_mapping_df.to_csv(home_dir+"data/gene/np_to_uniprot_mapping.csv", index=False, header=True, sep="\t")
np_to_uniprot_mapping_df

,NCBI_protein_accession,UniProtKB_protein_accession
0,NP_000005.3,Q6PN97
1,NP_000005,P01023
2,NP_000006.2,A4Z6T7
3,NP_000006.2,Q9H220
4,NP_000006,A4Z6T7
...,...,...
1282962,NP_999997,D3ZS96
1282963,NP_999998.1,A0A8I6A837
1282964,NP_999998,M0RDQ2
1282965,NP_999999.1,D3ZNH9
